In [ ]:
import os, pandas as pd, networkx as nx, numpy as np, statsmodels.api as sm
from sqlalchemy import create_engine
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt, seaborn as sns

## parameters

In [ ]:
yeari, yearf = '2024', '2024'
weeki, weekf = '18', '31'

In [ ]:
di = datetime.strptime(f'{yeari}-{weeki}-1', "%Y-%W-%w").date()
df = datetime.strptime(f'{yearf}-{weekf}-1', "%Y-%W-%w").date() + timedelta(6)
ds = [di+timedelta(dt) for dt in range((df-di).days+1)]
daylist = ds
print(di, 'until', df)

In [ ]:
cdef = 'tl7_10m'# 'tl5_10m' 'tl6_10m' 'tl7_10m' 'tl8_10m' 'tl8_60m'
cdef_alt = '16m_10min'# tl5: 62 ... tl7: 16   tl8: 8

## load data

In [ ]:
# load stadium meta data
stadname_data = pd.read_csv('output/00_stadname_data.csv')
aid2name = {aid: name for aid, name in zip(stadname_data.area_id, stadname_data.area_name)}

aid2city = {aid: city for aid, city in sorted({
    1: 'München',
    2: 'Berlin',
    3: 'Leipzig',
    4: 'Frankfurt am Main',
    10: 'Dortmund',
    11: 'Stuttgart',
    15: 'Köln',
    24: 'Düsseldorf',
    25: 'Hamburg',
    28: 'Gelsenkirchen',

    16: 'Berlin',
    29: 'Hamburg',
    41: 'München',
    64: 'Frankfurt am Main',
}.items(), key=lambda item: item[1])}
city2aid = {city: aid for aid, city in [(aid, city) for aid, city in aid2city.items()][::-1]}

In [ ]:
# n_c cities: load total contact numbers numbers for cities
data = pd.read_csv(f'output/00_ncontacts_cities_{cdef}.csv')
data['day'] = [d.date() for d in pd.to_datetime(data.day)]

# n_d stadiums: load did numbers for stadiums
panelstad_data2 = pd.read_csv('output/00_panelstad_data2.csv')
panelstad_data2['day'] = [d.date() for d in pd.to_datetime(panelstad_data2.day)]

# load mass event data
match_data = pd.read_csv('output/00_event_data.csv')
match_data['day'] = [d.date() for d in pd.to_datetime(match_data.day)]

## plot stadium $n_c$ and $n_d$

In [ ]:
# n_c contact numbers
data2 = data.copy(deep=True)
data2 = data2[['day','city','area_id','ncontacts_2']]
data2.rename(columns={'ncontacts_2':'n'}, inplace=True)
data2['which'] = 'contacts'

# n_d device numbers
panelstad_data3 = panelstad_data2.copy(deep=True)
panelstad_data3['city'] = [aid2city[aid] for aid in panelstad_data3.area_id]
panelstad_data3.rename(columns={'ndids':'n'}, inplace=True)
panelstad_data3['which'] = 'devices'

# join both
joint_data = pd.concat([data2, panelstad_data3])
joint_data['n'] = [np.log10(n) if n>0 else 0 for n in joint_data.n]
euro24_stadiums = [1,2,3,4,10,11,15,24,25,28]
add_stadiums = [16,29,41,64]# additional stadiums not involved in EURO 2024
joint_data = joint_data[joint_data.area_id.isin(euro24_stadiums + add_stadiums)]
joint_data

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

# Plot the lines on two facets
g = sns.relplot(
    data=joint_data,
    x="day", y="n",
    hue="which", hue_order=['devices','contacts'],
    row="area_id", row_order=list(aid2city.keys()),# size="which",# col="align",
    kind="line",# palette=palette,# size_order=[False, True], 
    height=2, aspect=4., facet_kws=dict(sharex=False),
    alpha=.5
)

axes = g.axes
holis = [date(2024,5,9), date(2024,5,20)]
for ax_row, (aid, city) in zip(axes, aid2city.items()):
    for ax in ax_row:
        ax.set_ylabel('log10(count)')
        ax.set_title(f"{city.split(' ')[0]} - {aid2name[aid]}")
        #daylist = [di+timedelta(dt) for dt in range(1+(df-di).days)]
        ax.set_xticks(daylist)
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for t,d in enumerate(daylist)])#, rotation=90)
        #ax.set_xlim([di-timedelta(7), df+timedelta(7)])
        ax.set_ylim([0., 3.])
        matches_here = match_data[match_data.area_id==aid]
        for day, match in zip(matches_here.day, matches_here.match):
            if day in daylist:
                ax.text(day, 3., match, rotation=90, ha='center', va='top')

plt.savefig(f'plots/fig1_{cdef_alt}/02_panelcontactsstad.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/02_panelcontactsstad.pdf', bbox_inches='tight')
plt.show()

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

# Plot the lines on two facets
g = sns.relplot(
    data=joint_data[joint_data.area_id.isin(list(aid2city.keys())[:7])],
    x="day", y="n",
    hue="which", hue_order=['devices','contacts'],
    row="area_id", row_order=list(aid2city.keys())[:7],# size="which",# col="align",
    kind="line",# palette=palette,# size_order=[False, True], 
    height=2, aspect=4., facet_kws=dict(sharex=False),
    alpha=.5
)

axes = g.axes
holis = [date(2024,5,9), date(2024,5,20)]
for ax_row, (aid, city) in zip(axes, [(aid, city) for (aid, city) in aid2city.items()][:7]):
    for ax in ax_row:
        ax.set_ylabel('log10(count)')
        ax.set_title(f"{city.split(' ')[0]} - {aid2name[aid]}")
        #daylist = [di+timedelta(dt) for dt in range(1+(df-di).days)]
        ax.set_xticks(daylist)
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for t,d in enumerate(daylist)])#, rotation=90)
        #ax.set_xlim([di-timedelta(7), df+timedelta(7)])
        ax.set_ylim([0., 3.])
        matches_here = match_data[match_data.area_id==aid]
        for day, match in zip(matches_here.day, matches_here.match):
            if day in daylist:
                ax.text(day, 3., match, rotation=90, ha='center', va='top')

plt.savefig(f'plots/fig1_{cdef_alt}/02_panelcontactsstad_1.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/02_panelcontactsstad_1.pdf', bbox_inches='tight')
plt.show()

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

# Plot the lines on two facets
g = sns.relplot(
    data=joint_data[joint_data.area_id.isin(list(aid2city.keys())[7:])],
    x="day", y="n",
    hue="which", hue_order=['devices','contacts'],
    row="area_id", row_order=list(aid2city.keys())[7:],# size="which",# col="align",
    kind="line",# palette=palette,# size_order=[False, True], 
    height=2, aspect=4., facet_kws=dict(sharex=False),
    alpha=.5
)

axes = g.axes
holis = [date(2024,5,9), date(2024,5,20)]
for ax_row, (aid, city) in zip(axes, [(aid, city) for (aid, city) in aid2city.items()][7:]):
    for ax in ax_row:
        ax.set_ylabel('log10(count)')
        ax.set_title(f"{city.split(' ')[0]} - {aid2name[aid]}")
        #daylist = [di+timedelta(dt) for dt in range(1+(df-di).days)]
        ax.set_xticks(daylist)
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for t,d in enumerate(daylist)])#, rotation=90)
        #ax.set_xlim([di-timedelta(7), df+timedelta(7)])
        ax.set_ylim([0., 3.])
        matches_here = match_data[match_data.area_id==aid]
        for day, match in zip(matches_here.day, matches_here.match):
            if day in daylist:
                ax.text(day, 3., match, rotation=90, ha='center', va='top')

plt.savefig(f'plots/fig1_{cdef_alt}/02_panelcontactsstad_2.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/02_panelcontactsstad_2.pdf', bbox_inches='tight')
plt.show()